In [1]:
import hddm
from patsy import dmatrix  # for generation of (regression) design matrices
import numpy as np         # for basic matrix operations
from pandas import Series  # to manipulate data-frames generated by hddm
%%capture output

/home/jasongong/.local/lib/python3.6/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [2]:
n_subjects = 100
trials_per_level = 20 # and per stimulus

In [3]:
level0 = {'v':.0, 'a':2.8, 't':.4, 'sv':0.05, 'z':.5, 'sz':0, 'st':0.2}
level1 = {'v':.1, 'a':2.8, 't':.4, 'sv':0.05, 'z':.5, 'sz':0, 'st':0.2}
level2 = {'v':-0.1, 'a':2.8, 't':.4, 'sv':0.05, 'z':.5, 'sz':0, 'st':0.2}
level3 = {'v':.2, 'a':2.8, 't':.4, 'sv':0.05, 'z':.5, 'sz':0, 'st':0.2}
level4 = {'v':.3, 'a':2.8, 't':.4, 'sv':0.05, 'z':.5, 'sz':0, 'st':0.2}

In [4]:
#import Pool
from multiprocessing import Pool
#Define a worker — a function which will be executed in parallel
def gen_worker(x):
    np.random.seed(x)
    print('Now generating No.', x)
    sim_data, sim_params = hddm.generate.gen_rand_data({'level0': level0,
                                             'level1': level1,
                                             'level2': level2,
                                             'level3': level3,
                                             'level4': level4},
                                             size=trials_per_level,
                                             subjs=n_subjects)
    print('Finished generating No.', x)

    return sim_data

num_processors = 4
#Create a pool of processors
p=Pool(processes = num_processors)

#get them to work in parallel
gen_data = p.map(gen_worker,[i for i in range(200)])


Now generating No. 26
Now generating No. 39
Now generating No. 13
Now generating No. 0
Finished generating No. 13
Now generating No. 14
Finished generating No. 26
Now generating No. 27
Finished generating No. 39
Finished generating No. 0
Now generating No. 40
Now generating No. 1
Finished generating No. 14
Now generating No. 15
Finished generating No. 40
Now generating No. 41
Finished generating No. 27
Now generating No. 28
Finished generating No. 1
Now generating No. 2
Finished generating No. 28
Now generating No. 29
Finished generating No. 15
Now generating No. 16
Finished generating No. 41
Now generating No. 42
Finished generating No. 2
Now generating No. 3
Finished generating No. 16
Now generating No. 17
Finished generating No. 29
Now generating No. 30
Finished generating No. 3
Now generating No. 4
Finished generating No. 42
Now generating No. 43
Finished generating No. 30
Now generating No. 31
Finished generating No. 17
Now generating No. 18
Finished generating No. 4
Now generatin

Now generating No. 197
Finished generating No. 158
Now generating No. 159
Finished generating No. 184
Now generating No. 185
Finished generating No. 171
Now generating No. 172
Finished generating No. 197
Now generating No. 198
Finished generating No. 159
Now generating No. 160
Finished generating No. 185
Now generating No. 186
Finished generating No. 172
Now generating No. 173
Finished generating No. 198
Now generating No. 199
Finished generating No. 160
Now generating No. 161
Finished generating No. 186
Now generating No. 187
Finished generating No. 173
Now generating No. 174
Finished generating No. 199
Finished generating No. 161
Now generating No. 162
Finished generating No. 187
Now generating No. 188
Finished generating No. 174
Now generating No. 175
Finished generating No. 162
Now generating No. 163
Finished generating No. 188
Now generating No. 189
Finished generating No. 175
Now generating No. 176
Finished generating No. 163
Now generating No. 164
Finished generating No. 189
Now

In [ ]:
#import Pool
from multiprocessing import Pool
#Define a worker — a function which will be executed in parallel
def worker(x):
    m = hddm.HDDM(gen_data[x], depends_on={'v': 'condition'})
    m.find_starting_values()
    print('Now fitting No.', x)
    m.sample(500, burn=20)
    print('Finished fitting No.', x)
    return m

num_processors = 4
#Create a pool of processors
p=Pool(processes = num_processors)

#get them to work in parallel
output = p.map(worker,[i for i in range(200)])

/home/jasongong/.local/lib/python3.6/site-packages/scipy/optimize/optimize.py:1985: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/home/jasongong/.local/lib/python3.6/site-packages/scipy/optimize/optimize.py:1985: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/home/jasongong/.local/lib/python3.6/site-packages/scipy/optimize/optimize.py:1985: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/home/jasongong/.local/lib/python3.6/site-packages/scipy/optimize/optimize.py:1985: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


Now fitting No. 13
Now fitting No. 26
Now fitting No. 39
Now fitting No. 0
 [-----------------100%-----------------] 500 of 500 complete in 1541.6 secFinished fitting No. 26
 [-----------------99%----------------- ] 498 of 500 complete in 1545.8 secFinished fitting No. 13
 [-----------------100%-----------------] 500 of 500 complete in 1555.2 secFinished fitting No. 0
 [-----------------100%-----------------] 501 of 500 complete in 1558.2 secFinished fitting No. 39
Now fitting No. 27
 [                  0%                  ] 2 of 500 complete in 2.7 secNow fitting No. 14
 [                  0%                  ] 4 of 500 complete in 7.9 seccNow fitting No. 1
 [                  1%                  ] 7 of 500 complete in 16.9 secNow fitting No. 40
 [-----------------98%----------------- ] 494 of 500 complete in 1556.8 secFinished fitting No. 40
 [-----------------99%----------------- ] 495 of 500 complete in 1570.1 secFinished fitting No. 1
 [-----------------99%----------------- ] 499 

In [ ]:
import pickle

with open("gen_data.txt", "wb") as fp:   #Pickling
    pickle.dump(gen_data, fp)

with open("output.txt", "wb") as fp:   #Pickling
    pickle.dump(output, fp)

In [ ]:
output.show()